# Projeto Final 2 - Construção de uma RAG utilizando LangChain

## Aluna:

* **Nome:** Thaís Rezende e Barbosa
* **E-mail:** trb@cesar.school

## Elaboração do projeto

Este projeto tem como objetivo alimentar a LLM através de uma RAG com os dados de salários dos servidores públicos do município de Cachoeira de Minas/MG em dezembro/2024. Os dados foram obtidos no Portal da Transparência (https://portal.sgpcloud.net:9117/transparencia/) e exportados para PDF.

Critérios de avaliação:

* Escolha do Documento
* Splitting do Documento
* Criação de Vector Store
* Retrieval
* Geração de Respostas

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [2]:
! pip install langchain-community langchain_openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.4 MB/s eta 0:00:00


## Testando a LLM sem RAG

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [4]:
response = llm.invoke("Qual foi o salário do prefeito de Cachoeira de Minas/MG em dezembro de 2024?")
print(response.content)

Desculpe, mas não tenho acesso a informações específicas sobre salários de prefeitos ou dados atualizados além de outubro de 2023. Para obter informações sobre o salário do prefeito de Cachoeira de Minas/MG em dezembro de 2024, recomendo consultar o site oficial da prefeitura ou a câmara municipal da cidade, onde geralmente essas informações são publicadas.


## Retrieval Augmented Generation (RAG)

### Carregando Documentos - Loading

In [5]:
! pip install pypdf wget

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.9 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=12f02f9d5d8eea826b0b281f099edbd73d2950835693b7f2c0be495e47eafb34
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [23]:
from langchain_community.document_loaders import PyPDFLoader
import wget

# Obtendo o arquivo
url = 'https://nbviewer.org/github/thaisrezendeb/topicos_contemporaneos_202401/blob/main/Projeto%20final/Portal_Transp._Servidores.pdf'
wget.download(url, 'Portal_Transp._Servidores.pdf')

'Portal_Transp._Servidores.pdf'

In [24]:
# Fazendo a leitura do documento
loader = PyPDFLoader('/content/Portal_Transp._Servidores.pdf')
docs = loader.load()

len(docs[0].page_content)

3157

### Dividindo Documentos - Splitting/Chunking

In [25]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=500, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
len(all_splits)

391

### Indexando - Store

In [8]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

### Buscando e Recuperando Informações - Retrieve

In [9]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

### Gerando Respostas - Generate

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [18]:
for chunk in rag_chain.stream("Qual foi o salário do prefeito de Cachoeira de Minas/MG em dezembro de 2024?"):
    print(chunk, end="", flush=True)

O salário do prefeito de Cachoeira de Minas/MG em dezembro de 2024 foi de R$ 15.907,47.

In [19]:
for chunk in rag_chain.stream("Qual é o nome do prefeito?"):
    print(chunk, end="", flush=True)

O nome do prefeito é Dirceu D'Angelo de Faria.

## Conclusão

Este projeto teve como finalidade a construção de um RAG utilizando LangChain para fins de estudo. Utilizou-se para isto os dados atualizados da folha de pagamento da Prefeitura da Cachoeira de Minas/MG. Os dados foram obtidos no Portal da Transparência e são públicos, porém, dado que a LLM utilizada (gpt-4o-mini) teve seu treinamento finalizado antes da publicação deste material, ela não foi capaz de responder corretamente aos questionamentos referentes a um período mais recente.

As informações coletadas no Portal da Transparência foram exportadas para PDF para servirem de enriquecimento da LLM. O document foi dividido em *chunks* de tamanho 1000 e *overlap* de 500, resultando num total de 391 splits. Foi utilizada a biblioteca FAISS para vetorizar o conteúdo e gerar o *retriever*.

O sistema recebeu como template a instrução de atuar como um assistente de perguntas e respostas considerando o contexto apresentado. Neste caso o contexto foi o documento PDF contendo a folha de pagamento de dez/2024. As respostas são obtidas ao executar a *chain* configurada para que a LLM receba o contexto, a pergunta, e o template.

Ao final do processo, repetiu-se a pergunta para a LLM com RAG e ela foi capaz de responder ao questionamento. Portanto, o objetivo proposto para este projeto foi alcançado.